In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras import regularizers

from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt  
import math  
import cv2 


/home/arpit/anaconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'

nb_train_samples = 13000
# nb_validation_samples = 5000
epochs = 20
batch_size = 64

In [5]:
model_inception = applications.inception_v3.InceptionV3(weights = "imagenet", include_top=False, input_shape = (150, 150, 3))
# model_inception.summary()

In [6]:
datagen = ImageDataGenerator(rescale=1. / 255)  

In [7]:
train_generator = datagen.flow_from_directory(  
     train_data_dir,  
     target_size=(img_width, img_height),  
     batch_size=batch_size,  
     class_mode=None,  
     shuffle=False) 

nb_train_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))

Found 13000 images belonging to 30 classes.


In [8]:
%%time
bottleneck_features_train = model_inception.predict_generator(  
     train_generator, predict_size_train)

CPU times: user 2min 32s, sys: 5.18 s, total: 2min 37s
Wall time: 2min 20s


In [9]:
np.save('bottleneck_features_train.npy', bottleneck_features_train)  

In [10]:
bottleneck_features_train.shape

(13000, 3, 3, 2048)

In [11]:
datagen_top = ImageDataGenerator(rescale=1./255)  
generator_top = datagen_top.flow_from_directory(  
         train_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False) 

Found 13000 images belonging to 30 classes.


In [12]:
nb_train_samples = len(generator_top.filenames)  
num_classes = len(generator_top.class_indices) 

In [13]:
# load the bottleneck features saved earlier  
train_data = np.load('bottleneck_features_train.npy')

In [14]:
# get the class lebels for the training data, in the original order  
train_labels = generator_top.classes  

In [15]:
# convert the training labels to categorical vectors  
train_labels = to_categorical(train_labels, num_classes=num_classes)

In [21]:
model = Sequential()  
model.add(Flatten(input_shape=train_data.shape[1:]))  
model.add(Dense(4096, activation='relu'))  
model.add(Dense(2048, activation='relu'))  
model.add(Dense(1024, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))
# model.add(Dense(num_classes, activation='sigmoid'))
   
model.compile(optimizer='adam',  
              loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
history = model.fit(train_data, train_labels,  
          epochs=5,  
          batch_size=batch_size)
                      
#           validation_data=(validation_data, validation_labels))

Epoch 1/5
13000/13000 [==============================] - 13s 1ms/step - loss: 15.2024 - acc: 0.0536
Epoch 2/5
13000/13000 [==============================] - 12s 924us/step - loss: 15.2564 - acc: 0.0535
Epoch 3/5
13000/13000 [==============================] - 12s 924us/step - loss: 15.2564 - acc: 0.0535
Epoch 4/5
13000/13000 [==============================] - 12s 924us/step - loss: 15.2564 - acc: 0.0535
Epoch 5/5
13000/13000 [==============================] - 12s 924us/step - loss: 15.2564 - acc: 0.0535


In [ ]:
plt.figure(1)  

# summarize history for accuracy  

plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  

# summarize history for loss  

plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()

In [17]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
test_generator = test_datagen.flow_from_directory(
    directory="data/test",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 6000 images belonging to 1 classes.


In [19]:
nb_test_samples = len(test_generator.filenames)

In [ ]:
%%time
bottleneck_features_test = model_inception.predict_generator(test_generator)

In [ ]:
bottleneck_features_test.shape

In [ ]:
np.save('bottleneck_features_test.npy', bottleneck_features_test) 

In [ ]:
test_data = np.load('bottleneck_features_test.npy') 

In [ ]:
pred = model.predict(test_data)

In [ ]:
pred.shape

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
predictions = list(predicted_class_indices)

In [ ]:
filenames=test_generator.filenames

In [ ]:
results=pd.DataFrame({"filename":filenames,
                      "label":predictions})
results.to_csv("results/results_inception_bottleneck.csv",index=False)